In [1]:
# como acessar s3?


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

#Criando Sessão Spark
spark = SparkSession.builder \
.appName('XPTO')\
.getOrCreate()

# diretorio da pasta onde foram salvos os JSONs extraidos
df = spark.read.json('jsonDF/')

In [3]:
# padronizar tudo em lowercase
df = df.withColumn('tweet_text', lower(col('tweet_text')))

In [4]:
# tirando os filhos do bolso e o iguinho do lulinha
df = df.filter(locate('iguinho', 'tweet_text') == 0)
df = df.filter(locate('eduardo', 'tweet_text') == 0)
df = df.filter(locate('carlos','tweet_text') == 0)
df = df.filter(locate('flávio', 'tweet_text') == 0)

In [5]:
# primeiro definir de quem estão falando
df = df.withColumn('lula', when(locate('lula', 'tweet_text') == 0, None).otherwise(locate('lula', 'tweet_text')))\
.withColumn('lulinha', when(locate('lulinha', 'tweet_text') == 0, None).otherwise(locate('lulinha', 'tweet_text')))\
.withColumn('presidiario', when(locate('presidiário', 'tweet_text') == 0, None).otherwise(locate('presidiário', 'tweet_text')))\
.withColumn('bolsonaro', when(locate('bolsonaro', 'tweet_text') == 0, None).otherwise(locate('bolsonaro', 'tweet_text')))\
.withColumn('bozo', when(locate('bozo', 'tweet_text') == 0, None).otherwise(locate('bozo', 'tweet_text')))\
.withColumn('biroliro', when(locate('biroliro', 'tweet_text') == 0, None).otherwise(locate('biroliro', 'tweet_text')))\
.withColumn('menor', least(col('lula'), col('lulinha'), col('presidiario'), col('bolsonaro'), col('bozo'), col('biroliro')))\
.withColumn('PalavraChave', expr("""substr(tweet_text,menor,4)"""))\
.withColumn('Candidato', when(col('PalavraChave') == 'lula', "lula")\
            .when(col('PalavraChave') == 'luli', 'lula')\
            .when(col('PalavraChave') == 'pres', 'lula')\
            .when(col('PalavraChave') == 'bols', 'bolsonaro')\
            .when(col('PalavraChave') == 'bozo', 'bolsonaro')\
            .when(col('PalavraChave') == 'biro', 'bolsonaro'))\
.withColumn('Sentimento', when(col('PalavraChave') == 'lula', "neutro")\
            .when(col('PalavraChave') == 'luli', 'neutro')\
            .when(col('PalavraChave') == 'pres', 'negativo')\
            .when(col('PalavraChave') == 'bols', 'neutro')\
            .when(col('PalavraChave') == 'bozo', 'negativo')\
            .when(col('PalavraChave') == 'biro', 'negativo')\
           .otherwise('neutro'))\
.drop('lula','bolsonaro','lulinha','presidiario','biroliro','bozo','menor', 'PalavraChave')

In [10]:
df.select('tweet_text', 'Sentimento').filter((locate('iguinho', 'tweet_text') != 0) & (col('Sentimento') == 'neutro')).show(truncate=80)

+----------+----------+
|tweet_text|Sentimento|
+----------+----------+
+----------+----------+



In [7]:
# filtrar negativo e positivo
df = df.withColumn('Sentimento', when(col('tweet_text').contains("melhor"), 'positivo')\
            .when(col('tweet_text').contains("maravilhoso"), 'positivo')\
            .when(col('tweet_text').contains("otimo"), 'positivo')\
            .when(col('tweet_text').contains("ótimo"), 'positivo')\
            .when(col('tweet_text').contains("salvar"), 'positivo')\
            .when(col('tweet_text').contains("orgulho"), 'positivo')\
            .when(col('tweet_text').contains("honra"), 'positivo')\
            .when(col('tweet_text').contains("posse"), 'positivo')\
            .when(col('tweet_text').contains("esperança"), 'positivo')\
            .when(col('tweet_text').contains("confio"), 'positivo')\
            .when(col('tweet_text').contains("acredito"), 'positivo')\
            .when(col('tweet_text').contains("querido"), 'positivo')\
            .when(col('tweet_text').contains("nosso"), 'positivo')\
            .when(col('tweet_text').contains("meu presidente"), 'positivo')\
            .when(col('tweet_text').contains("feliz"), 'positivo')\
            .when(col('tweet_text').contains("contente"), 'positivo')\
            .when(col('tweet_text').contains("comemorar"), 'positivo')\
            .when(col('tweet_text').contains("patriotas"), 'positivo')\
            .when(col('tweet_text').contains("humanitário"), 'positivo')\
            .when(col('tweet_text').contains("humanitario"), 'positivo')\
            .when(col('tweet_text').contains("patriotas"), 'positivo')\
            .when(col('tweet_text').contains("🇧🇷"), 'positivo')\
              #negativos
            .when(col('tweet_text').contains("lixo"), 'negativo')\
            .when(col('tweet_text').contains("burro"), 'negativo')\
            .when(col('tweet_text').contains("idiota"), 'negativo')\
            .when(col('tweet_text').contains("bosta"), 'negativo')\
            .when(col('tweet_text').contains("otario"), 'negativo')\
            .when(col('tweet_text').contains("otário"), 'negativo')\
            .when(col('tweet_text').contains("minion"), 'negativo')\
            .when(col('tweet_text').contains("ladrao"), 'negativo')\
            .when(col('tweet_text').contains("ladrão"), 'negativo')\
            .when(col('tweet_text').contains("roub"), 'negativo')\
            .when(col('tweet_text').contains("matou"), 'negativo')\
            .when(col('tweet_text').contains("odeio"), 'negativo')\
            .when(col('tweet_text').contains("merda"), 'negativo')\
            .when(col('tweet_text').contains("nojo"), 'negativo')\
            .when(col('tweet_text').contains("canalha"), 'negativo')\
            .when(col('tweet_text').contains("bandido"), 'negativo')\
            .when(col('tweet_text').contains("porra"), 'negativo')\
            .when(col('tweet_text').contains("bozo"), 'negativo')\
            .when(col('tweet_text').contains("preso"), 'negativo')\
            .when(col('tweet_text').contains("bananinha"), 'negativo')\
            .when(col('tweet_text').contains("gado"), 'negativo')\
            .when(col('tweet_text').contains("analfabeto"), 'negativo')\
            .when(col('tweet_text').contains("corrupto"), 'negativo')\
            .when(col('tweet_text').contains("corrupção"), 'negativo')\
            .when(col('tweet_text').contains("estimação"), 'negativo')\
            .when(col('tweet_text').contains("hipócrita"), 'negativo')\
            .when(col('tweet_text').contains("hipocrita"), 'negativo')\
            .when(col('tweet_text').contains("cachaceiro"), 'negativo')\
            .when(col('tweet_text').contains("sociopata"), 'negativo')\
            .when(col('tweet_text').contains("psicopata"), 'negativo')\
            .when(col('tweet_text').contains("genocida"), 'negativo')\
            .when(col('tweet_text').contains("golpista"), 'negativo')\
            .when(col('tweet_text').contains("nazista"), 'negativo')\
            .when(col('tweet_text').contains("triste"), 'negativo')\
            .when(col('tweet_text').contains("descontente"), 'negativo')\
            .when(col('tweet_text').contains("infeliz"), 'negativo')\
            .when(col('tweet_text').contains("desumano"), 'negativo')\
            .when(col('tweet_text').contains("desgoverno"), 'negativo')\
            .when(col('tweet_text').contains("presidiário"), 'negativo').otherwise('neutro'))


In [8]:
#anilisar
#df.select('tweet_text','Candidato','Sentimento').show(50, truncate=80)
df.groupBy("Sentimento").count().show()

+----------+-----+
|Sentimento|count|
+----------+-----+
|    neutro|  622|
|  negativo|  184|
|  positivo|  125|
+----------+-----+



In [19]:
from pyspark.sql.types import TimestampType

df = df.withColumn('tweet_date', col('tweet_date').cast('timestamp'))
df = df.withColumn("year", date_format(col("tweet_date"), "yyyy"))\
        .withColumn("month", date_format(col("tweet_date"), "MM"))\
        .withColumn("day", date_format(col("tweet_date"), "dd"))
df.show()

+-------------------+-------------------+--------------------+---------+----------+----+-----+---+
|                 id|         tweet_date|          tweet_text|Candidato|Sentimento|year|month|day|
+-------------------+-------------------+--------------------+---------+----------+----+-----+---+
|1597789029847764992|2022-11-30 00:06:04|lista dos que dev...|bolsonaro|  negativo|2022|   11| 30|
|1597789116581969920|2022-11-30 00:06:25|sacanagem isso!\n...|bolsonaro|  positivo|2022|   11| 30|
|1597789196932247554|2022-11-30 00:06:44|a pergunta é ? “ ...|     lula|  negativo|2022|   11| 30|
|1597789231383912448|2022-11-30 00:06:52|lula será o prime...|     lula|    neutro|2022|   11| 30|
|1597789239634448384|2022-11-30 00:06:54|dr beijamim arrol...|     lula|    neutro|2022|   11| 30|
|1597789311126372352|2022-11-30 00:07:11|o povo está pedin...|     lula|    neutro|2022|   11| 30|
|1597789448603074560|2022-11-30 00:07:44|bolsonaro vai ped...|bolsonaro|    neutro|2022|   11| 30|
|159778953